# Input Generation Support for HEPROW and GRAVEL
This notebook takes root TTrees and turns them into the correctly formatted .phs and .rsp files for the the GRAVEL and HEPROW unfolding programs.  It will also generate the input neutron spectrum (for a simulation) to use in comparison.  

Root must be installed and PyRoot enabled for this notebook to work.

First, load the necessary libraries, set the appropriate environment variables, and compile the C++ support macros from the instructions @ https://bitbucket.org/berkeleylab/nsd-rootscripts/wiki/LibraryCompilation.

Load python dependencies:

In [1]:
from ROOT import gROOT
import numpy as np

gROOT.ProcessLine('HistogramWriter writer;')
gROOT.ProcessLine('HistogramOperations ops;')

0L

## Specify the user inputs

#### Input File Locations
dataPath = the absolute location of where the root files are located <br/>
phsName = file name for the pulse height spectra TTree <br/>
rspName = file name for the response matrix TTree <br/>

In [3]:
dataPath="/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Simulated/PHS/"
phsName="16MeVBe/16MeVBeRun00046.root"

rspName="ResponseMatrices/NeuSim18mil.root"

/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Simulated/PHS/16MeVBe/16MeVBeRun00046.root
/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Simulated/PHS/ResponseMatrices


#### Output File Location
outPath = the absolute path of where to place the generated HEPROW input files

In [4]:
outPath="/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Simulated/PHS/"

#### Pulse Height Binning
phsBins = the number of bins to use for the binning of the pulse height spectrum <br/>
phsEmin = the minimum Energy to use for the binning of the pulse height spectrum <br/>
phsEmax = the maximum Energy to use for the binning of the pulse height spectrum <br/>

In [5]:
phsBins=56
phsEmin=0.0
phsEmax=14

#### Response Matrix Binning

uniform = boolean variable indicating whether uniform bins or variable bins are input

##### If uniform bins are desired:
rspEwidth = the width of bins to use for the binning of the response matrix energy <br/>
rspEmin = the minimum Energy to use for the binning of the response matrix  <br/>
rspEmax = the maximum Energy to use for the binning of the response matrix  <br/>
rspLwidth = the width of bins to use for the binning of the response matrix light <br/>
rspLmin = the minimum Light to use for the binning of the response matrix  <br/>
rspLmax = the maximum Light to use for the binning of the response matrix  <br/>

##### If variable bins are desired:
rspEbins = the bin edges for the binning of the response matrix energy <br/>
respLbins = the bin edges for the binning of the response matrix light <br/>

In [6]:
uniform = True

# Must be defined if uniform = True
rspEwidth=10
rspEmin=0.0
rspEmax=40.0
rspLwidth=0.25
rspLmin=0.0
rspLmax=30.0

# Must be defined if uniform = False
rspEbins=[0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0]
rspEbins=[0.0,1.0,2.0,3.0,4.0,5.0,5.5,6.0,6.5,7.0,7.5,8.0,8.5,9.0,9.5,10.0,10.5,11.0,11.5,12.0,12.5,13.0,13.5,14.0,14.5,15.0,15.5,16.0,16.5,17.0,17.5,18.0,18.5,19.0,19.5,20.0,20.5,21.0]
rspEbins=[0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,5.5,6.0,6.5,7.0,7.5,8.0,8.5,9.0,9.5,10.0,10.5,11.0,11.5,12.0,12.5,13.0,13.5,14.0,14.5,15.0,15.5,16.0,16.5,17.0,17.5,18.0,18.5,19.0,19.5,20.0]
rspLbins=np.arange(rspLmin,rspLmax,rspLwidth)

## Load PHS Data File
If this needs to be reran, the notebook kernel needs to be reset

In [7]:
gROOT.ProcessLine('TFile f("{}")'.format(dataPath+phsName))

47366329344096L

## Create the .phs File

In [ ]:
gROOT.ProcessLine('simEventTree->Draw("getNetTargetLight()>>n1({},{},{})")'.format(phsBins,phsEmin,phsEmax))
gROOT.ProcessLine('writer.PhToHEPROW(n1,"{}phs_101")'.format(outPath))

## Rebin Statistically

In [ ]:
gROOT.ProcessLine('ops.rebinStatistically(n1,10,false)->Draw()')
gROOT.ProcessLine('writer.PhToHEPROW(ops.rebinStatistically(n1,10,false),"{}stat_10_phs_25")'.format(outPath))

### Smear the PHS and Create the .phs File
Only do this for a simulated spectrum

In [ ]:
gROOT.ProcessLine('ops.rebinStatistically(ops.skewedGausSmear(n1,1.00065e-05,0.0897338,8.65248e-07),10,false)->Draw()')
gROOT.ProcessLine('writer.PhToHEPROW(ops.rebinStatistically(ops.skewedGausSmear(n1,1.00065e-05,0.0897338,8.65248e-07),10,false),"{}stat_10_smearedPHS_25")'.format(outPath))

In [ ]:
gROOT.ProcessLine('ops.skewedGausSmear(n1,1.00065e-05,0.0897338,8.65248e-07)->Draw()')
gROOT.ProcessLine('writer.PhToHEPROW(ops.skewedGausSmear(n1,1.00065e-05,0.0897338,8.65248e-07),"{}smearedPHS_25")'.format(outPath))

## Ouput the Initial Spectrum

For comparison to the unfolded spectrum.  Only useful for simulated TTrees

In [ ]:
gROOT.ProcessLine('primary->Draw("nEn>>n2(40,0,20)")')
gROOT.ProcessLine('writer.Th1ToAscii(n2,"{}inputSpectrum_50")'.format(outPath))

Create an input .flu file if desired:

In [ ]:
gROOT.ProcessLine('writer.PhToHEPROW(n2,"{}sim.flu")'.format(outPath))

## Load Response Matrix Data File
If this needs to be reran, the notebook kernel needs to be reset. The .phs and Initial Spectrum cells above may not work once this is run.

In [ ]:
gROOT.ProcessLine('SimulationManipulation sm("{}",0)'.format(dataPath+rspName))

## Create the .rsp File

In [ ]:
# Create the bin structures
if uniform:
    rspEbins=np.arange(rspEmin,rspEmax,rspEwidth)
    rspEbins=np.append(rspEbins,rspEmax)
    print rspEbins
    rspLbins=np.arange(rspLmin,rspLmax,rspLwidth)
    rspLbins=np.append(rspLbins,rspLmax)
    print rspLbins
    gROOT.ProcessLine('const Int_t EBINS = {}; const Int_t LBINS = {};'.format(len(rspEbins)-1,len(rspLbins)-1))
    gROOT.ProcessLine('Double_t eEdges[EBINS + 1] = {}{}{};'.format("{",", ".join(str(e) for e in rspEbins),"}"))
    gROOT.ProcessLine('Double_t lEdges[LBINS + 1] = {}{}{};'.format("{",", ".join(str(e) for e in rspLbins),"}"))
    gROOT.ProcessLine('axis1 = TAxis(EBINS,eEdges);')
    gROOT.ProcessLine('axis2 = TAxis(LBINS,lEdges);')
else:
    gROOT.ProcessLine('const Int_t EBINS = {}; const Int_t LBINS = {};'.format(len(rspEbins)-1,len(rspLbins)-1))
    gROOT.ProcessLine('Double_t eEdges[EBINS + 1] = {}{}{};'.format("{",", ".join(str(e) for e in rspEbins),"}"))
    gROOT.ProcessLine('Double_t lEdges[LBINS + 1] = {}{}{};'.format("{",", ".join(str(e) for e in rspLbins),"}"))
    gROOT.ProcessLine('axis1 = TAxis(EBINS,eEdges);')
    gROOT.ProcessLine('axis2 = TAxis(LBINS,lEdges);')

# Create the Histogram and output file
gROOT.ProcessLine('TH2* hist=sm.getNormalizedResponseMatrix(axis1,axis2)')
#gROOT.ProcessLine('TH2* hist=sm.getNormalizedResponseMatrix(1.0, 0.05, 0, 22)')
#gROOT.ProcessLine('hist->Draw()')
gROOT.ProcessLine('hist->Draw("colz")')
gROOT.ProcessLine('writer.ResponseToHEPROW(hist,"{}resp_25_1")'.format(outPath))

### Smear the Response Matrix and Create the .rsp File
Only do this for a simulated response matrix

In [ ]:
gROOT.ProcessLine('TH2* smearHist = ops.skewedGausSmearMatrix(hist,1.00065e-05,0.0897338,8.65248e-07)')
gROOT.ProcessLine('smearHist->Draw("colz")')
gROOT.ProcessLine('writer.ResponseToHEPROW(smearHist,"{}smearedResp_25_1")'.format(outPath))

## Mik.inp Support

These lines are useful to develop the mik.inp file:

In [ ]:
print "{}".format(len(rspEbins)-1)
for i in range(1,len(rspEbins)):
    print "{:.1f}                                     right boundary of {:1d} energy interval".format(rspEbins[i],i)